In [ ]:
#pinecone
#env
import os
from dotenv import load_dotenv
load_dotenv()
#client
from pinecone import Pinecone
pinecone_client=Pinecone(api_key=os.getenv("secret_key_pinecone"))
#create index
index_name="post"
from pinecone import ServerlessSpec
import time
if not pinecone_client.has_index(index_name):pinecone_client.create_index(name=index_name,dimension=1024,metric="cosine",spec=ServerlessSpec(cloud='aws',region='us-east-1'))
while not pinecone_client.describe_index(index_name).status['ready']:time.sleep(3)
index_client=pinecone_client.Index(index_name)
#data to insert
object_list=[{"id": "vec1", "text": "Apple is a popular fruit known for its sweetness and crisp texture and red color."},{"id": "vec2", "text": "The tech company Apple is known for its innovative products like the iPhone."},{"id": "vec3", "text": "Many people enjoy eating apples as a healthy snack."},{"id": "vec4", "text": "Apple Inc. has revolutionized the tech industry with its sleek designs and user-friendly interfaces."},{"id": "vec5", "text": "An apple a day keeps the doctor away, as the saying goes."},{"id": "vec6", "text": "Apple Computer Company was founded on April 1, 1976, by Steve Jobs, Steve Wozniak, and Ronald Wayne as a partnership."}]
text_list=inputs=[item['text'] for item in object_list]
#create embeddngs
embeddings=pinecone_client.inference.embed(inputs=text_list,model="multilingual-e5-large",parameters={"input_type":"passage","truncate":"END"})
#upsert embeddngs
records=[]
for object,embedding in zip(object_list,embeddings):records.append({"id":object['id'],"values":embedding['values'],"metadata":{'text':object['text']}})
index_client.upsert(vectors=records,namespace="example-namespace")
time.sleep(10)
print(index_client.describe_index_stats())
#search
query="Tell me about the health benefits of Apple."
query_embedding=pinecone_client.inference.embed(model="multilingual-e5-large",inputs=[query],parameters={"input_type":"query"})
results=index_client.query(namespace="example-namespace",vector=query_embedding[0].values,top_k=3,include_values=False,include_metadata=True)
print(results)
#rerank
documents = [{"id": x["id"], "text": x["metadata"]["text"]} for x in results["matches"]]
rerank_results = pinecone_client.inference.rerank(model="bge-reranker-v2-m3",query=query,documents=documents,top_n=3,return_documents=True)
print(rerank_results)
#delete index
pinecone_client.delete_index(index_name)
print("index deleted")


In [ ]:
#chromadb
import chromadb
chromadb_client=chromadb.PersistentClient(path="/Users/atom/Downloads/chromadb")
collection=chromadb_client.get_or_create_collection(name="my_collection",metadata={"hnsw:space":"cosine"})
documents=["This is a document about pineapple","This is a document about oranges","this is a document about steve"]
ids=["id1", "id2","id3"]
metadatas=[{"type":"beach","health":2},{"type":"normal","health":1},{"type":"human"}]
collection.upsert(documents=documents,ids=ids,metadatas=metadatas)
query=["This is a query document about human"]
results = collection.query(query_texts=query,n_results=3)
print(results)
chromadb_client.delete_collection(name="my_collection")